In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Đường dẫn tới thư mục chứa dữ liệu
DUONG_DAN_DU_LIEU = "D:/code_things/do an cuoi ki mon may hoc/du doan diem cuoi ki/wecode.data/"

def tai_va_tien_xu_li_du_lieu():
    """Tải và tiền xử lý dữ liệu từ WeCode"""
    try:
        logger.info("Đang đọc file dữ liệu nộp bài...")
        # Đọc file Excel chứa dữ liệu nộp bài
        df = pd.read_csv(os.path.join(DUONG_DAN_DU_LIEU, "annonimized.csv"))
        
        # Chỉ giữ lại các cột quan trọng
        cols_to_keep = ['concat(\'it001\', username)', 'is_final', 'pre_score', 'coefficient']
        df = df[cols_to_keep]
        
        # Đổi tên cột 
        df = df.rename(columns={
            'concat(\'it001\', username)': 'username',
            'pre_score': 'diem',
            'coefficient': 'he_so_tre'
        })
        df['diem'] = df['diem'] / 100
        logger.info("Tính toán đặc trưng tổng hợp...")
        aggregation = {
            'diem': ['mean', 'max', 'count'],
            'is_final': 'sum',
            'he_so_tre': 'mean'
        }
        
        dac_trung_sinh_vien = df.groupby("username").agg(aggregation)
        
        # Làm phẳng các cột multi-index
        dac_trung_sinh_vien.columns = ['_'.join(col).strip() for col in dac_trung_sinh_vien.columns.values]
        
        # Chuẩn hóa tên cột
        column_mapping = {
            'diem_mean': 'diem_trung_binh',
            'diem_max': 'diem_cao_nhat',
            'diem_count': 'so_lan_nop',
            'is_final_sum': 'so_bai_diem_cuoi',
            'he_so_tre_mean': 'he_so_tre'
        }
        dac_trung_sinh_vien = dac_trung_sinh_vien.rename(columns=column_mapping)
        
        logger.info("Tiền xử lý dữ liệu hoàn tất!")
        return dac_trung_sinh_vien
        
    except Exception as e:
        logger.error(f"Lỗi khi tải và tiền xử lý dữ liệu: {str(e)}", exc_info=True)
        return None

def huan_luyen_mo_hinh(dac_trung_sinh_vien, duong_dan_diem_thuc_hanh):
    """Huấn luyện mô hình hồi quy tuyến tính"""
    try:
        logger.info("Đang tải điểm thực hành...")
        # Tải điểm thực hành thực tế
        diem_thuc_hanh = pd.read_csv(duong_dan_diem_thuc_hanh)
        
        # Ghép dữ liệu đặc trưng với điểm thực hành
        logger.info("Ghép dữ liệu...")
        du_lieu_day = pd.merge(
            dac_trung_sinh_vien.reset_index(),
            diem_thuc_hanh,
            left_on="username",
            right_on="hash",
            how='inner'
        )
        
        # Kiểm tra NaN ban đầu
        logger.info(f"Số dòng có NaN trước khi xử lý:\n{du_lieu_day.isna().sum()}")
        
        # Ép kiểu cột TH về float
        logger.info("Ép kiểu cột TH về float...")
        du_lieu_day['TH'] = pd.to_numeric(du_lieu_day['TH'], errors='coerce')
        
        # Kiểm tra lại NaN sau khi ép kiểu
        logger.info(f"Số NaN ở cột TH sau khi ép kiểu: {du_lieu_day['TH'].isna().sum()}")
        
        # Loại bỏ NaN
        logger.info("Loại bỏ các dòng có NaN...")
        du_lieu_day = du_lieu_day.dropna()
        logger.info(f"Số dòng sau khi loại NaN: {len(du_lieu_day)}")
        
        # Chọn các đặc trưng và biến mục tiêu
        features = [
            'diem_trung_binh', 'diem_cao_nhat', 'so_lan_nop',
            'so_bai_diem_cuoi', 'he_so_tre'
        ]
        
        X = du_lieu_day[features]
        y = du_lieu_day['TH']  # Cột điểm thực hành trong file th-public.csv
        
        # Chuẩn hóa dữ liệu
        logger.info("Chuẩn hóa dữ liệu...")
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Chia tập train/test
        logger.info("Chia tập train/test...")
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=42)
        
        # Huấn luyện mô hình
        logger.info("Đang huấn luyện mô hình...")
        mo_hinh = LinearRegression()
        mo_hinh.fit(X_train, y_train)
        
        # Đánh giá mô hình
        y_pred = mo_hinh.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        logger.info(f"Độ chính xác R^2: {r2:.4f}")
        
        return mo_hinh, scaler, features
        
    except Exception as e:
        logger.error(f"Lỗi khi huấn luyện mô hình: {str(e)}", exc_info=True)
        return None, None, None


def du_doan_diem(mo_hinh, scaler, features, dac_trung_sinh_vien):
    """Dự đoán điểm cho tất cả sinh viên"""
    try:
        logger.info("Đang chuẩn bị dữ liệu để dự đoán...")
        # Chuẩn bị dữ liệu đầu vào
        X = dac_trung_sinh_vien[features]
        
        # Chuẩn hóa dữ liệu
        X_scaled = scaler.transform(X)
        
        # Dự đoán điểm
        logger.info("Đang dự đoán điểm...")
        diem_du_doan = mo_hinh.predict(X_scaled)
        
        # Làm tròn về các mốc 0.5
        diem_du_doan = np.round(diem_du_doan * 2) / 2
        
        # Tạo dataframe kết quả
        ket_qua = pd.DataFrame({
            'MaSV': dac_trung_sinh_vien.index,
            'DiemThucHanh_du_doan': diem_du_doan
        })
        
        return ket_qua
        
    except Exception as e:
        logger.error(f"Lỗi khi dự đoán điểm: {str(e)}", exc_info=True)
        return None



def main():
    try:
        # Bước 1: Tải và tiền xử lý dữ liệu
        logger.info("Đang tải và tiền xử lý dữ liệu...")
        dac_trung_sinh_vien = tai_va_tien_xu_li_du_lieu()
        
        if dac_trung_sinh_vien is None:
            logger.error("Không thể tiến hành do lỗi tiền xử lý dữ liệu")
            return
        
        # Bước 2: Huấn luyện mô hình
        logger.info("Đang huấn luyện mô hình...")
        duong_dan_diem_thuc_hanh = os.path.join(DUONG_DAN_DU_LIEU, "th-public.csv")
        mo_hinh, scaler, features = huan_luyen_mo_hinh(dac_trung_sinh_vien, duong_dan_diem_thuc_hanh)
        
        if mo_hinh is None:
            logger.error("Không thể tiến hành do lỗi huấn luyện mô hình")
            return
        
        # Bước 3: Dự đoán điểm cho tất cả sinh viên
        logger.info("Đang dự đoán điểm...")
        ket_qua = du_doan_diem(mo_hinh, scaler, features, dac_trung_sinh_vien)
        
        if ket_qua is not None:
            # Bước 4: Lưu kết quả
            duong_dan_ket_qua = os.path.join(DUONG_DAN_DU_LIEU, "ket_qua_du_doan.csv")
            ket_qua.to_csv(duong_dan_ket_qua, index=False)
            logger.info(f"Đã lưu kết quả dự đoán vào: {duong_dan_ket_qua}")
        
    except Exception as e:
        logger.error(f"Lỗi chương trình chính: {str(e)}", exc_info=True)

if __name__ == "__main__":
    main()

INFO:__main__:Đang tải và tiền xử lý dữ liệu...
INFO:__main__:Đang đọc file dữ liệu nộp bài...
INFO:__main__:Tính toán đặc trưng tổng hợp...
INFO:__main__:Tiền xử lý dữ liệu hoàn tất!
INFO:__main__:Đang huấn luyện mô hình...
INFO:__main__:Đang tải điểm thực hành...
INFO:__main__:Ghép dữ liệu...
INFO:__main__:Số dòng có NaN trước khi xử lý:
username            0
diem_trung_binh     0
diem_cao_nhat       0
so_lan_nop          0
so_bai_diem_cuoi    0
he_so_tre           0
hash                0
TH                  6
dtype: int64
INFO:__main__:Ép kiểu cột TH về float...
INFO:__main__:Số NaN ở cột TH sau khi ép kiểu: 8
INFO:__main__:Loại bỏ các dòng có NaN...
INFO:__main__:Số dòng sau khi loại NaN: 753
INFO:__main__:Chuẩn hóa dữ liệu...
INFO:__main__:Chia tập train/test...
INFO:__main__:Đang huấn luyện mô hình...
INFO:__main__:Độ chính xác R^2: 0.1986
INFO:__main__:Đang dự đoán điểm...
INFO:__main__:Đang chuẩn bị dữ liệu để dự đoán...
INFO:__main__:Đang dự đoán điểm...
INFO:__main__:Đã lưu k